In [ ]:
import os
import shutil
import json

trainer_k_mapping = {
    0: 20,
    1: 40,
    2: 80,
    3: 160,
    4: 320,
    5: 640
}
dict_size = 16384
activation_dim = 2304

# Assume current directory is the one containing "trainer_*" folders.
base_dir = os.getcwd()

# Iterate over each trainer folder
for trainer_folder in os.listdir(base_dir):
    trainer_path = os.path.join(base_dir, trainer_folder)
    if os.path.isdir(trainer_path) and trainer_folder.startswith("trainer_"):
        try:
            trainer_id = int(trainer_folder.split("_")[-1])
        except ValueError:
            print(f"Skipping {trainer_folder}: unable to parse trainer ID.")
            continue

        # Path where ae.pt is currently stored inside the redundant folder structure.
        src_ae = os.path.join(trainer_path, "peft_sae_full_finetune", "rank_16", "ae.pt")
        # New destination: directly under the trainer folder.
        dst_ae = os.path.join(trainer_path, "ae.pt")

        if os.path.exists(src_ae):
            # Move ae.pt from its old location to the trainer folder.
            shutil.move(src_ae, dst_ae)
            print(f"Moved: {src_ae} -> {dst_ae}")
        else:
            raise ValueError(f"Warning: {src_ae} does not exist.")

        # Remove the redundant folder structure.
        redundant_dir = os.path.join(trainer_path, "peft_sae_full_finetune")
        if os.path.exists(redundant_dir):
            shutil.rmtree(redundant_dir)
            print(f"Removed directory: {redundant_dir}")

        # Build the config with the correct "k" value.

        config = {
            "trainer": {
                "trainer_class": "TopKTrainer",
                "dict_class": "AutoEncoderTopK",
                "lr": 0.0003,
                "steps": 244140,
                "auxk_alpha": 0.03125,
                "warmup_steps": 1000,
                "decay_start": 195312,
                "threshold_beta": 0.999,
                "threshold_start_step": 1000,
                "seed": 0,
                "activation_dim": activation_dim,
                "dict_size": dict_size,
                "k": trainer_k_mapping[trainer_id],
                "device": "cuda:0",
                "layer": 12,
                "lm_name": "google/gemma-2-2b",
                "wandb_name": "TopKTrainer-google/gemma-2-2b-resid_post_layer_12_trainer_0",
                "submodule_name": "resid_post_layer_12"
            },
            "buffer": {
                "d_submodule": 2304,
                "io": "out",
                "n_ctxs": 244,
                "ctx_len": 1024,
                "refresh_batch_size": 4,
                "out_batch_size": 2048,
                "device": "cuda:0"
            }
        }


        # Write the config.json file next to the ae.pt file.
        config_path = os.path.join(trainer_path, "config.json")
        with open(config_path, "w") as f:
            json.dump(config, f, indent=4)
        print(f"Created config file at: {config_path}")

In [ ]:
import os
import shutil

new_folder = "gemma-2-2b_top_k_width-2pow14_date-0107"
existing_folder = "gemma-2-2b"
subfolder = "resid_post_layer_12"

# Rename existing folder to subfolder
os.rename(existing_folder, subfolder)

# Create the new folder
os.makedirs(new_folder, exist_ok=True)

# Move subfolder into new_folder
shutil.move(subfolder, os.path.join(new_folder, subfolder))

print(f"Renamed '{existing_folder}' to '{subfolder}'")
print(f"Created new folder: '{new_folder}'")
print(f"Moved '{subfolder}' into '{new_folder}'")

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_large_folder(
    folder_path=".",
    # path_in_repo="",
    repo_id="adamkarvonen/new_kl_finetunes",
    repo_type="model",
    allow_patterns=["*.json", "*.pt"],
    ignore_patterns=[".DS_Store"],
)